### **TODO and Notes**

 - [x] Make dataframes
 - [x] Convert datetimes
 - [x] rename date columns
 - [ ] Find Nans
 - [ ] Re-freq and fill blanks
 - [ ] Turn logs into daily totals
 - [ ] Save backup CSVs
 - [ ] Combine dataframe
 
 **Notes**
Sample for iterating through different offsets 
```python
df["Input"].corr(df["Output"].shift(-1), method = 'pearson', min_periods = 1) #1
```
and more iteration 
```python
 xcov_monthly = [crosscorr(datax, datay, lag=i) for i in range(12)]
```
from [here](https://stackoverflow.com/questions/33171413/cross-correlation-time-lag-correlation-with-pandas)


## Imports, data, checks

In [1]:
import numpy as np
import requests
import pandas as pd
from urllib.request import urlopen
import json
from bokeh.models import CategoricalColorMapper, NumeralTickFormatter, HoverTool
from bokeh.models import ColumnDataSource, Grid, LinearAxis, Plot, VBar
from bokeh.plotting import output_notebook, figure
from bokeh.io import reset_output, show, output_file
from bokeh.layouts import column, row

The vaccine, cases, and deaths source data were relatively easy to grab diretly from the [Larimer county dashboard](https://www.larimer.org/health/communicable-disease/coronavirus-covid-19/larimer-county-positive-covid-19-numbers#/app?tab=risk) as the CSVs download through urls.

In [2]:
larimer_vac = pd.read_csv('https://speedtest.larimer.org/covid/index.php?file=vaccinations&csv')
larimer_vac.name = 'larimer_vac'

larimer_cases = pd.read_csv('https://speedtest.larimer.org/covid/cases.csv', parse_dates=['ReportedDate'])
larimer_cases.name = 'larimer_cases'

larimer_deaths = pd.read_csv('https://larimer-county-data-lake.s3-us-west-2.amazonaws.com/Public/covid/covid_deaths.csv?t=1631890252549')
larimer_deaths.name = 'larimer_deaths'

# setting names may have been a bad idea or at least pointless

The hospitalization data was much more tricky (at least finding a simple solution was tricky) I spent several hours in webscraping research and attempts purgatory. I checked BeautifulSoup, html5lib, lxml, etc. in multiple combinations and none of them had straightforward solutions because the table for hospitalizations is actually rendered through javascript so there is nothing to scrape without actually clicking the buttons. I started down the Selenium and phantomjs path but it seemed like a nightmare. I found this lifesaving article at [Towards Data Science](https://towardsdatascience.com/data-science-skills-web-scraping-javascript-using-python-97a29738353f) which shows how to find specific XHR request urls in the browser developer tools. The requested URL for the rendered table is a pretty vanilla json and not behind any authorization so there is a pretty clean way to get to it. Praise Satan I didn't have to use Selenium.  

In [3]:
url = 'https://larimer-county-data-lake.s3-us-west-2.amazonaws.com/Public/covid/covid_patient_trend.json?t=1632506827395'

response = urlopen(url)
json_data = response.read().decode('utf-8', 'replace')

d = json.loads(json_data)
larimer_hosp = pd.json_normalize(d['data'])
larimer_hosp.name = 'larimer_hosp'


So now we have all of our dataframs

In [4]:
display(larimer_vac)

display(larimer_cases)

display(larimer_deaths)

display(larimer_hosp)

,Date,daily number of doses received by Larimer County residents,total number of doses recevied by residents,daily number of residents receiving first dose,total number of residents receiving first dose,daily number of residents vaccinated,total number of residents vaccinated,daily number of 70+ vaccinated,total number of 70+ vaccinated,daily number of 70+ at least one dose,...,daily number of Latinx residents vaccinated,total of Latinx residents vaccinated,daily number of White non-Latinx residents vaccinated,total of White non-Latinx residents vaccinated,daily number of non-White non-Latinx residents vaccinated,total of non-White non-Latinx residents vaccinated,dailyUnknown,totalUnknown,daily_additional_doses,total_additional_doses
0,12/14/2020,32,32,24,24,8,8,1.0,1,0.0,...,0.0,0,8,8,0.0,0,0.0,0,0,0
1,12/15/2020,18,50,8,32,10,18,0.0,1,1.0,...,0.0,0,9,17,1.0,1,NaN,0,0,0
2,12/16/2020,311,361,301,333,10,28,0.0,1,1.0,...,1.0,1,8,25,1.0,2,0.0,0,0,0
3,12/17/2020,985,1346,967,1300,18,46,1.0,2,10.0,...,0.0,1,18,43,0.0,2,0.0,0,0,0
4,12/18/2020,1039,2385,1026,2326,13,59,2.0,4,13.0,...,0.0,1,12,55,1.0,3,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,10/20/2021,712,456583,109,230278,108,217758,5.0,34538,7.0,...,10.0,12645,77,184407,8.0,12301,13.0,8405,513,26215
311,10/21/2021,400,456983,96,230374,103,217861,6.0,34544,4.0,...,10.0,12655,70,184477,9.0,12310,14.0,8419,224,26439
312,10/22/2021,1027,458010,214,230588,138,217999,5.0,34549,14.0,...,15.0,12670,93,184570,8.0,12318,22.0,8441,692,27131
313,10/23/2021,1661,459671,208,230796,75,218074,10.0,34559,22.0,...,1.0,12671,63,184633,4.0,12322,7.0,8448,1383,28514


,CaseCount,ReportedDate,Sex,Age,Type,City
0,1,2020-03-09,Female,52.0,Confirmed,Johnstown
1,2,2020-03-15,Male,49.0,Confirmed,Fort Collins
2,3,2020-03-17,Female,53.0,Confirmed,Fort Collins
3,4,2020-03-17,Female,94.0,Confirmed,Loveland
4,5,2020-03-18,Male,49.0,Confirmed,Fort Collins
...,...,...,...,...,...,...
39682,40655,2021-10-26,Female,55.0,Confirmed,Loveland
39683,40656,2021-10-26,Female,71.0,Confirmed,Windsor
39684,40657,2021-10-26,Male,71.0,Confirmed,Loveland
39685,40658,2021-10-26,Female,79.0,Confirmed,Loveland


,death_id,death_date,age,gender,city,case_status,count
0,a0U5w00000edbfjEAA,2020-03-09,91,Female,Loveland,Probable,1
1,a0U5w00000edbfiEAA,2020-03-13,95,Female,Loveland,Probable,2
2,a0U5w00000edbfOEAQ,2020-03-15,90,Female,Loveland,Probable,3
3,a0U5w00000edbfJEAQ,2020-03-25,87,Female,Fort Collins,Confirmed,4
4,a0U5w00000edbfMEAQ,2020-03-25,74,Female,Loveland,Confirmed,5
...,...,...,...,...,...,...,...
309,a0U5w00000foc2QEAQ,2021-10-15,39,Female,Laporte,Confirmed,310
310,a0U5w00000foc2REAQ,2021-10-16,73,Female,Fort Collins,Confirmed,311
311,a0U5w00000foc2SEAQ,2021-10-19,66,Male,Wellington,Confirmed,312
312,a0U5w00000focfkEAA,2021-10-21,73,Female,Fort Collins,Confirmed,313


,Date,admission_count,kpi_admits_indicator,inpatient_count,kpi_patient_indicator,inpatient_count_pct_change
0,2020-03-31T00:00:00.000Z,NaN,NaN,47,0,NaN
1,2020-04-01T00:00:00.000Z,NaN,NaN,46,0,NaN
2,2020-04-02T00:00:00.000Z,NaN,NaN,46,0,NaN
3,2020-04-03T00:00:00.000Z,2.0,0.0,46,0,NaN
4,2020-04-04T00:00:00.000Z,1.0,0.0,42,0,NaN
...,...,...,...,...,...,...
389,2021-10-19T00:00:00.000Z,12.0,0.0,92,1,9.523810
390,2021-10-20T00:00:00.000Z,10.0,0.0,98,1,15.294118
391,2021-10-21T00:00:00.000Z,14.0,0.0,100,1,19.047619
392,2021-10-22T00:00:00.000Z,8.0,0.0,97,1,6.593407


This looks like pretty good start. We'll have to make all the datetimes match and the **hospitalization** and **vaccine** data are daily totals while the **death** and **case counts** data is a case log (a row for each case) so we'll have to do some grouping to get that to match, that will come later.

## Explore, clean, manipulate

In [5]:
dfs = [larimer_vac, larimer_deaths, larimer_cases, larimer_hosp]

def get_obj_col():
    for df in dfs:
        print(list(df.select_dtypes(['object']).columns))

get_obj_col()

['Date']
['death_id', 'death_date', 'gender', 'city', 'case_status']
['Sex', 'Type', 'City']
['Date']


---
I did this and don't like it
```python

dfs = [larimer_vac, larimer_deaths, larimer_cases, larimer_hosp]
df_names = ['larimer_vac', 'larimer_deaths', 'larimer_cases', 'larimer_hosp']


def get_obj_col():
    for df in dfs:
        obj_cols.append(list(df.select_dtypes(['object']).columns))
    zip(df_names, dfs)
    
obj_cols = []
get_obj_col()
zipped_list = zip(df_names, obj_cols)
print(tuple(zipped_list)
```
---

In [6]:
print(larimer_cases.dtypes)
print(larimer_hosp.dtypes)

CaseCount                int64
ReportedDate    datetime64[ns]
Sex                     object
Age                    float64
Type                    object
City                    object
dtype: object
Date                           object
admission_count               float64
kpi_admits_indicator          float64
inpatient_count                 int64
kpi_patient_indicator           int64
inpatient_count_pct_change    float64
dtype: object


Convert date columns from each df to datetimes

In [7]:
larimer_vac['Date'] = pd.to_datetime(larimer_vac['Date']).dt.tz_localize(None)
larimer_deaths['Date'] = pd.to_datetime(larimer_deaths['death_date']).dt.tz_localize(None)
larimer_cases['Date'] = pd.to_datetime(larimer_cases['ReportedDate']).dt.tz_localize(None)
larimer_hosp['Date'] = pd.to_datetime(larimer_hosp['Date']).dt.tz_localize(None)



```pd.to_datetime``` was sufficient for most of the dfs but the hospital data was TZ aware and I wanted all of them to match so had to add the ```.dt.tz_localize(None)``` 

In [8]:
def check_date_type():
    for df in dfs:
        print(list(df.select_dtypes(['datetime64']).columns))

check_date_type()

['Date']
['Date']
['ReportedDate', 'Date']
['Date']


# BOOKMARK

In [6]:
# create daily cases from case log
daily_cases = larimer_cases.groupby(['ReportedDate']).count().reset_index()

display(daily_cases)
display(daily_cases.dtypes)
print(f"Total case check {daily_cases['CaseCount'].sum()}")
display(daily_cases.describe()) 

,ReportedDate,CaseCount,Sex,Age,Type,City
0,2020-03-09,1,1,1,1,1
1,2020-03-15,1,1,1,1,1
2,2020-03-17,2,2,2,2,2
3,2020-03-18,1,1,1,1,1
4,2020-03-19,2,2,2,2,2
...,...,...,...,...,...,...
558,2021-09-25,73,73,73,73,73
559,2021-09-26,92,92,91,92,92
560,2021-09-27,81,81,81,81,81
561,2021-09-28,97,97,97,97,97


ReportedDate    datetime64[ns]
CaseCount                int64
Sex                      int64
Age                      int64
Type                     int64
City                     int64
dtype: object

Total case check 35943


,CaseCount,Sex,Age,Type,City
count,563.000000,563.000000,563.000000,563.000000,563.000000
mean,63.841918,63.841918,63.683837,63.841918,63.841918
std,65.053481,65.053481,64.824479,65.053481,65.053481
min,1.000000,1.000000,1.000000,1.000000,1.000000
25%,15.000000,15.000000,15.000000,15.000000,15.000000
50%,41.000000,41.000000,41.000000,41.000000,41.000000
75%,93.000000,93.000000,93.000000,93.000000,93.000000
max,342.000000,342.000000,337.000000,342.000000,342.000000


In [7]:
daily_cases.head()

,ReportedDate,CaseCount,Sex,Age,Type,City
0,2020-03-09,1,1,1,1,1
1,2020-03-15,1,1,1,1,1
2,2020-03-17,2,2,2,2,2
3,2020-03-18,1,1,1,1,1
4,2020-03-19,2,2,2,2,2


In [8]:
daily_cases['ReportedDate'] = pd.to_datetime(daily_cases['ReportedDate'])

In [9]:
daily_cases.set_index('ReportedDate', inplace=True)

In [10]:
daily_cases.dtypes

CaseCount    int64
Sex          int64
Age          int64
Type         int64
City         int64
dtype: object

In [11]:
# daily_cases.index = pd.to_datetime(daily_cases.index)
# daily_cases = daily_cases.resample("1D").mean()
# daily_cases


In [12]:
daily_cases_filled = daily_cases.asfreq('D',fill_value=0)

In [13]:
daily_cases_filled.head(10)

,CaseCount,Sex,Age,Type,City
ReportedDate,,,,,
2020-03-09,1,1,1,1,1
2020-03-10,0,0,0,0,0
2020-03-11,0,0,0,0,0
2020-03-12,0,0,0,0,0
2020-03-13,0,0,0,0,0
2020-03-14,0,0,0,0,0
2020-03-15,1,1,1,1,1
2020-03-16,0,0,0,0,0
2020-03-17,2,2,2,2,2


**Try this**

```python
x.dt = pd.to_datetime(x.dt)
```
One-liner using mostly @ayhan's ideas while incorporating stack/unstack and fill_value

```python
x.set_index(
    ['dt', 'user']
).unstack(
    fill_value=0
).asfreq(
    'D', fill_value=0
).stack().sort_index(level=1).reset_index()
```
**or this might be better**
```python
s.asfreq('D'))
```


In [ ]:
daily_cases.dtypes

In [ ]:
larimer_cases.ReportedDate.dtype

In [16]:
larimer_vac.shape

(289, 21)

In [17]:
len(larimer_vac)

289

In [18]:
larimer_vac.count()

Date                                                          289
daily number of doses received by Larimer County residents    289
total number of doses recevied by residents                   289
daily number of residents receiving first dose                289
total number of residents receiving first dose                289
daily number of residents vaccinated                          289
total number of residents vaccinated                          289
daily number of 70+ vaccinated                                287
total number of 70+ vaccinated                                289
daily number of 70+ at least one dose                         287
total number of 70+ at least one dose                         289
daily number of Latinx residents vaccinated                   288
total of Latinx residents vaccinated                          289
daily number of White non-Latinx residents vaccinated         289
total of White non-Latinx residents vaccinated                289
daily numb

In [19]:
larimer_vac.describe()

,daily number of doses received by Larimer County residents,total number of doses recevied by residents,daily number of residents receiving first dose,total number of residents receiving first dose,daily number of residents vaccinated,total number of residents vaccinated,daily number of 70+ vaccinated,total number of 70+ vaccinated,daily number of 70+ at least one dose,total number of 70+ at least one dose,daily number of Latinx residents vaccinated,total of Latinx residents vaccinated,daily number of White non-Latinx residents vaccinated,total of White non-Latinx residents vaccinated,daily number of non-White non-Latinx residents vaccinated,total of non-White non-Latinx residents vaccinated,dailyUnknown,totalUnknown,daily_additional_doses,total_additional_doses
count,289.000000,289.000000,289.000000,289.000000,289.000000,289.000000,287.000000,289.000000,287.000000,289.000000,288.000000,289.000000,289.000000,289.000000,287.000000,289.000000,284.000000,289.000000,289.000000,289.000000
mean,1509.747405,255347.754325,786.636678,142692.273356,741.121107,121049.377163,119.675958,24810.221453,124.470383,28481.242215,42.670139,6252.211073,628.349481,104526.923875,41.738676,6277.757785,29.306338,3992.484429,41.930796,781.373702
std,1416.479307,156611.864631,860.448141,80837.783985,844.618435,82505.692463,259.417495,13061.885998,252.175621,11132.556133,47.644358,4688.573179,731.028342,70115.060711,52.422217,4747.997720,29.682320,3078.927324,172.484063,2350.682395
min,2.000000,31.000000,2.000000,23.000000,0.000000,8.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,437.000000,89429.000000,210.000000,59471.000000,172.000000,29839.000000,6.000000,15348.000000,8.000000,28315.000000,12.750000,1122.000000,129.000000,26971.000000,11.000000,1042.000000,12.000000,704.000000,0.000000,0.000000
50%,916.000000,326580.000000,375.000000,183775.000000,332.000000,155229.000000,13.000000,32521.000000,16.000000,34187.000000,27.000000,7254.000000,252.000000,135900.000000,21.000000,7617.000000,20.000000,4458.000000,0.000000,0.000000
75%,2414.000000,395130.000000,1037.000000,211091.000000,1046.000000,199611.000000,83.000000,33792.000000,78.000000,34914.000000,53.000000,10918.000000,895.000000,170686.000000,47.000000,11039.000000,35.000000,6968.000000,0.000000,0.000000
max,6678.000000,436317.000000,4496.000000,227338.000000,4166.000000,214184.000000,1751.000000,34347.000000,1586.000000,35723.000000,312.000000,12289.000000,3617.000000,181593.000000,243.000000,11979.000000,159.000000,8323.000000,1504.000000,12118.000000


In [20]:
print(larimer_vac.Date.dtype)


object


In [21]:
real_date = pd.to_datetime(larimer_vac.Date)
real_date

0     2020-12-14
1     2020-12-15
2     2020-12-16
3     2020-12-17
4     2020-12-18
         ...    
284   2021-09-24
285   2021-09-25
286   2021-09-26
287   2021-09-27
288   2021-09-28
Name: Date, Length: 289, dtype: datetime64[ns]

In [22]:
larimer_vac['Date'] = real_date
larimer_vac

,Date,daily number of doses received by Larimer County residents,total number of doses recevied by residents,daily number of residents receiving first dose,total number of residents receiving first dose,daily number of residents vaccinated,total number of residents vaccinated,daily number of 70+ vaccinated,total number of 70+ vaccinated,daily number of 70+ at least one dose,...,daily number of Latinx residents vaccinated,total of Latinx residents vaccinated,daily number of White non-Latinx residents vaccinated,total of White non-Latinx residents vaccinated,daily number of non-White non-Latinx residents vaccinated,total of non-White non-Latinx residents vaccinated,dailyUnknown,totalUnknown,daily_additional_doses,total_additional_doses
0,2020-12-14,31,31,23,23,8,8,1.0,1,0.0,...,0.0,0,8,8,0.0,0,0.0,0,0,0
1,2020-12-15,16,47,6,29,10,18,0.0,1,1.0,...,0.0,0,9,17,1.0,1,NaN,0,0,0
2,2020-12-16,311,358,300,329,11,29,0.0,1,1.0,...,1.0,1,9,26,1.0,2,0.0,0,0,0
3,2020-12-17,986,1344,969,1298,17,46,1.0,2,10.0,...,0.0,1,17,43,0.0,2,0.0,0,0,0
4,2020-12-18,1041,2385,1029,2327,12,58,2.0,4,14.0,...,0.0,1,12,55,0.0,2,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,2021-09-24,644,433618,187,226982,241,213835,13.0,34319,14.0,...,16.0,12257,196,181334,9.0,11956,20.0,8288,223,10097
285,2021-09-25,1659,435277,148,227130,151,213986,12.0,34331,30.0,...,14.0,12271,117,181451,10.0,11966,10.0,8298,1368,11465
286,2021-09-26,196,435473,47,227177,73,214059,2.0,34333,3.0,...,8.0,12279,54,181505,4.0,11970,7.0,8305,76,11541
287,2021-09-27,293,435766,110,227287,75,214134,7.0,34340,8.0,...,4.0,12283,54,181559,5.0,11975,12.0,8317,124,11665


In [23]:
larimer_vac.Date.dtype

dtype('<M8[ns]')

In [24]:
for col in larimer_vac.columns:
    print(col)

Date
daily number of doses received by Larimer County residents
total number of doses recevied by residents
daily number of residents receiving first dose
total number of residents receiving first dose
daily number of residents vaccinated
total number of residents vaccinated
daily number of 70+ vaccinated
total number of 70+ vaccinated
daily number of 70+ at least one dose
total number of 70+ at least one dose
daily number of Latinx residents vaccinated
total of Latinx residents vaccinated
daily number of White non-Latinx residents vaccinated
total of White non-Latinx residents vaccinated
daily number of non-White non-Latinx residents vaccinated
total of non-White non-Latinx residents vaccinated
dailyUnknown
totalUnknown
daily_additional_doses
total_additional_doses


In [25]:
larimer_vac['daily number of doses received by Larimer County residents'].dtype

dtype('int64')

In [26]:
# make a backup file from the source
larimer_vac.to_csv('larimer_vac_backup.csv')

In [27]:
larimer_vac[['Date','daily number of doses received by Larimer County residents']]

,Date,daily number of doses received by Larimer County residents
0,2020-12-14,31
1,2020-12-15,16
2,2020-12-16,311
3,2020-12-17,986
4,2020-12-18,1041
...,...,...
284,2021-09-24,644
285,2021-09-25,1659
286,2021-09-26,196
287,2021-09-27,293


## Quantify missing data

In [28]:
valid_entries = larimer_vac.count()
total_rows = len(larimer_vac.index)
missing_data = total_rows - valid_entries
missing_data

Date                                                          0
daily number of doses received by Larimer County residents    0
total number of doses recevied by residents                   0
daily number of residents receiving first dose                0
total number of residents receiving first dose                0
daily number of residents vaccinated                          0
total number of residents vaccinated                          0
daily number of 70+ vaccinated                                2
total number of 70+ vaccinated                                0
daily number of 70+ at least one dose                         2
total number of 70+ at least one dose                         0
daily number of Latinx residents vaccinated                   1
total of Latinx residents vaccinated                          0
daily number of White non-Latinx residents vaccinated         0
total of White non-Latinx residents vaccinated                0
daily number of non-White non-Latinx res

## Visualize

In [29]:
#lar_vac_data = ColumnDataSource(larimer_vac)

reset_output()
output_notebook()

x = larimer_vac['Date']
top = larimer_vac['daily number of doses received by Larimer County residents']

daily_vac_figure = figure(title="Daily Vaccinations",
                         x_axis_type="datetime")

daily_vac_figure.vbar(x=x,
               top=top,
               width=0.9)



show(daily_vac_figure)

Loading BokehJS ...

In [30]:
lar_vac_data = ColumnDataSource(larimer_vac)

reset_output()
output_notebook()

# x = lar_vac_data['Date']
# y = lar_vac_data('daily number of doses received by Larimer County residents')

daily_vac_figure = figure(title='Daily Vaccinations',
                         x_axis_type="datetime")

daily_vac_figure.line(x='Date',
                   y='daily number of doses received by Larimer County residents',
               source=lar_vac_data)



show(daily_vac_figure)

Loading BokehJS ...